In [1]:
%load_ext watermark
import json
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
#pip install tqdm
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
%matplotlib inline
import datetime
import collections
from datetime import datetime, date, time,timedelta
# настройка внешнего вида графиков в seaborn
sns.set_context(
    "notebook", 
    font_scale = 1.5,       
    rc = { 
        "figure.figsize" : (12, 9), 
        "axes.titlesize" : 18 
    }
)

import requests
from datetime import datetime, date, time
from datetime import timedelta

In [2]:
url = 'https://data.gov.ru/api/json/dataset/7708660670-proizvcalendar/version/20151123T183036/content'

In [11]:
resp = requests.get(url=url)
resp.content

b'<!DOCTYPE html>\n<!--[if IEMobile 7]><html class="iem7"  lang="ru" dir="ltr"><![endif]-->\n<!--[if lte IE 6]><html class="lt-ie9 lt-ie8 lt-ie7"  lang="ru" dir="ltr"><![endif]-->\n<!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8"  lang="ru" dir="ltr"><![endif]-->\n<!--[if IE 8]><html class="lt-ie9"  lang="ru" dir="ltr"><![endif]-->\n<!--[if (gte IE 9)|(gt IEMobile 7)]><!--><html  lang="ru" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# owl: http://www.w3.org/2002/07/owl# rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns# rss: http://purl.org/rss/1.0/ site: https://data.gov.ru/ns# dcat: http://www.w3.org/ns/dcat#"><!--<![endif]-->\n\n<head>\n  <meta charset="utf-8" />\n<link rel="shortcut 

In [76]:

from pprint import pprint

with open('./data/calendar/content.json', encoding="utf-8") as f:
    calendar_json = json.load(f)

#building calendar
cld_inp = pd.DataFrame([],columns = ['year','month','days'])
for k in calendar_json:
    month_num = 1
    for j in ['Январь','Февраль','Март','Апрель','Май','Июнь','Июль','Август','Сентябрь','Октябрь','Ноябрь','Декабрь']:
        cld_inp.loc[len(cld_inp)]=[int(k['Год/Месяц']),month_num,list(map(int, k[j].replace('*','').replace('+','').split(',')))]  
        month_num+=1
        
#calculating
cld_out = collections.Counter() #pd.DataFrame([],columns = ['week','wd'])
start_date = datetime.strptime("2018-22-1", "%Y-%W-%w")
end_date = datetime.strptime("2018-40-1", "%Y-%W-%w")
print(start_date)
delta_days = (end_date-start_date).days
shift_day = timedelta(1) ##shift = 1 day
day_week = 0
week_num = 0
act_date = start_date
for d in range(1,delta_days+1):
    day_week+=1
    if act_date.day in list(cld_inp.loc[(cld_inp['year']==act_date.year)&(cld_inp['month']==act_date.month)]['days'])[0]:
        print(act_date.day)
        cld_out[week_num] += 1      
    act_date = act_date + shift_day
    if day_week >= 7:
        week_num += 1
        day_week = 0

2018-05-28 00:00:00
2
3
9
10
11
12
16
17
23
24
30
1
7
8
14
15
21
22
28
29
4
5
11
12
18
19
25
26
1
2
8
9
15
16
22
23
29
30
Counter({2: 4, 0: 2, 1: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2})


In [72]:
cld_inp

,year,month,days
0,1999,1,"[1, 2, 3, 4, 6, 7, 9, 10, 16, 17, 23, 24, 30, 31]"
1,1999,2,"[6, 7, 13, 14, 20, 21, 27, 28]"
2,1999,3,"[6, 7, 8, 13, 14, 20, 21, 27, 28]"
3,1999,4,"[3, 4, 10, 11, 17, 18, 24, 25, 30]"
4,1999,5,"[1, 2, 3, 4, 8, 9, 10, 15, 16, 22, 23, 29, 30]"
5,1999,6,"[5, 6, 11, 12, 13, 14, 19, 20, 26, 27]"
6,1999,7,"[3, 4, 10, 11, 17, 18, 24, 25, 31]"
7,1999,8,"[1, 7, 8, 14, 15, 21, 22, 28, 29]"
8,1999,9,"[4, 5, 11, 12, 18, 19, 25, 26]"
9,1999,10,"[2, 3, 9, 10, 16, 17, 23, 24, 30, 31]"
